### Imports

In [ ]:
#!pip install sgt
#!pip install py-tlsh
#!pip install nilsimsa

from datetime import datetime
import string
import random
from random import randrange
import re
import tlsh
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

### Debug Functions

In [ ]:
def get_hash_tensor(hash,label):
    '''
    takes a hash and a label of the kind [x, x] and turns them into tensors
    '''
    data_max = len(ALL_CHARS)
    if label is "anom":
        label = [0, 1]
    elif(label is "norm"):
        label = [1, 0]
    else:
        print("label must be norm or anom !")
    list_x = [hash, label]  
    input, label = list_x
    char_to_int = dict((c, i) for i, c in enumerate(ALL_CHARS))
    input = [char_to_int[char] for char in hash]
    label = torch.FloatTensor(label)
    return torch.LongTensor(input), label

In [ ]:
def tensor_float_parser(tensor_str):
    '''
    parse the result of a model evaluation model(input)
    '''
    tensor_str = str(tensor_str)
    try:
        pat = r'.*?\[(.*)].*'            
        match = re.search(pat, tensor_str)
        target_str = match.group(1)  
        arr = [float(i) for i in re.findall(r"[-+]?\d*\.\d+|\d+", target_str)]
    except:# when the tensor_str is a label with [1. , 0.]
        arr = [int(s) for s in tensor_str.split() if s.isdigit()]
    return arr

In [ ]:

def toy_result_check(list_x):
    '''
    takes a tensor and a label (0,1) anomalie ....... (1,0) normal...and tests them on the model
    '''
    custom_loader = DataLoader(list_x,1)
    input, label = iter(custom_loader)
    input = input.to(device)
    result = model(input)
    
    arr_res = tensor_float_parser(result)
    arr_lab = tensor_float_parser(label)
    #case: anomalie
    if arr_lab[0] < arr_lab[1]:
        if arr_res[0] < arr_res[1]: #arr_lab[0] and arr_res[1] > arr_lab[1]:
            print("the model identified the anomaly correctly")
        else:
            print("Model could not identify the anomaly correctly or the label could be wrong: result{} label{}".format(arr_res,arr_lab))   
    #case: normal
    elif(arr_lab[0] > arr_lab[1]):
        if arr_res[0] > arr_res[1]: #arr_lab[0] and arr_res[1] < arr_lab[1]:
            print("the model identified the normal hash correctly")
        else:
            print("Model could not identify the normal input or the label could be wrong: result{} label{}".format(arr_res,arr_lab))  